<a href="https://colab.research.google.com/github/dinagustina/portofolio/blob/main/MITask/Portofolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#menyiapkan Library python
import yfinance as yf
import yfinance as yfin
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import sklearn
import datetime as dt
# from pandas_datareader import data as pdr
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

#melakukan pengambilan data menggunakan yahoo finance
df = yf.download(['ADRO.JK','ANTM.JK','BRIS.JK','BRPT.JK','CPIN.JK','EMTK.JK','ERAA.JK','EXCL.JK','ICBP.JK','INCO.JK','INDF.JK','INKP.JK','INTP.JK','ITMG.JK','JPFA.JK','KLBF.JK','MIKA.JK','MNCN.JK','PGAS.JK','PTBA.JK','SMGR.JK','TINS.JK','TLKM.JK','TPIA.JK','UNTR.JK','UNVR.JK','WIKA.JK'] ,start='2021-12-01', end='2022-11-29')
data =df['Close']
data.describe()
data.info()

#pre-processing data
# Identifikasi nilai hilang
missing_values = data.isnull().sum()
#visualisasi data
data.plot(figsize = (20,12))
plt.title('Pergerakan Harga Saham', fontsize = 20)
plt.ylabel('Harga Saham', fontsize = 20)
plt.xlabel('Date', fontsize = 20)
plt.show()

#menghitung return,expected return dan risiko pada saham
return_saham = data.pct_change()
expected_returns = return_saham.mean()
risiko = return_saham.var()
tabel1 = pd.DataFrame({'Expected Return':expected_returns,'Risiko':risiko})
#pemilihan saham efisien berdasarkan nilai Expected return positif
stock_symbols = ['ADRO.JK','ANTM.JK','BRIS.JK','BRPT.JK','CPIN.JK','EMTK.JK','ERAA.JK','EXCL.JK','ICBP.JK','INCO.JK','INDF.JK','INKP.JK','INTP.JK','ITMG.JK','JPFA.JK','KLBF.JK','MIKA.JK','MNCN.JK','PGAS.JK','PTBA.JK','SMGR.JK','TINS.JK','TLKM.JK','TPIA.JK','UNTR.JK','UNVR.JK','WIKA.JK']
print("Historical Stock Returns:")
for symbol in stock_symbols:
    print(f"Stock: {symbol}")
    print(expected_returns[symbol])
    print()
positive_return_stocks = [symbol for symbol in stock_symbols if (expected_returns[symbol]> 0).any()]
# Buat DataFrame baru hanya dengan saham-saham return positif
saham_efisien = expected_returns[positive_return_stocks]
print("Stocks with Positive Returns:")
print(saham_efisien)

#melakukan prediksi pada masing-masing saham
data1 = yf.download(['ADRO.JK','INCO.JK','ITMG.JK'],start='2021-12-01', end='2022-11-30')
data1 = data1['Close']

#pembagian data training dan data testing
from pandas.core.construction import construct_1d_arraylike_from_scalar
train = data1[data1.index < pd.to_datetime("2022-08-01", format='%Y-%m-%d')]
test = data1[data1.index > pd.to_datetime("2022-08-01", format='%Y-%m-%d')]
plt.plot(train, color = "black")
plt.plot(test, color = "red")
plt.ylabel('HARGA SAHAM JII')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.title("Train/Test split for Data JII")
plt.show()
#prediksi harga saham dilakukan pada masing-masing indeks saham,

#dibawah ini merupakan bentuk formula prediksi SVR pada saham BBNI
yfin.pdr_override()
spy = pdr.get_data_yahoo(['INCO.JK'],start='2021-12-01', end='2022-11-29')
data2 = spy [['Close']]
#Pre-Processing data
x = data2.iloc[:,0].values
y = data2.iloc[:,-1].values
x = x.reshape(-1,1)
y = y.reshape(-1,1)

# Define the percentage of data for training
train_percentage = 0.7  # You can adjust this value based on your preference
# Calculate the index where the split should occur
split_index = int(len(x) * train_percentage)
# Split the data into training and testing sets
x_train, x_test = x[:split_index], x[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

#menentukan parameter dan kernel terbaik
my_param_grid = {'C':[0.001,0.01,0,0.1,10,100,1000], 'epsilon': [0.001,0.01,0,10,100,1000],'kernel':['rbf','linear']}
GridSearchCV(estimator=SVR(),param_grid= my_param_grid, refit = True, verbose=3, cv=3)
grid_rbf = GridSearchCV(estimator=SVR(),param_grid= my_param_grid, refit = True, verbose=3, cv=3)

#melatih model pada data training
grid_rbf.fit(x_train, y_train)

#mendapatkan parameter terbaik pada data pelatihan
print("Parameter Terbaik: ", grid_rbf.best_params_)

#melakukan pengujian model pada data testing
ypred = grid_rbf.predict(x_test)

#plot hasil prediksi pada data testing
plt.figure(figsize=(10,6))
plt.plot(y_test, label='Harga Aktual', color='blue')
plt.plot(ypred, label='Harga Prediksi', color='red')
plt.xlabel('Days', fontsize=10)
plt.ylabel('Close Price', fontsize=10)
plt.title('Harga Aktual vs Harga Prediksi', fontsize=15)
plt.legend()
plt.show()

#menyiapkan tabel perbandingan data aktual dan data prediksi
y_test = y_test.reshape(-1)
ypred = ypred.reshape(-1)
df = pd.DataFrame({'Data Aktual':y_test,'Data Prediksi':ypred})

# Calculate performance metrics
mse = mean_squared_error(y_test, ypred)
rmse = np.sqrt(mse)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error", rmse)